<a href="https://colab.research.google.com/github/divyansh212/AI-MODELS-/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset

torch.manual_seed(50)
df=pd.read_csv('fmnist_small.csv')
x=df.iloc[:,1:].values
y=df.iloc[:,0].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train=x_train/255.0
x_test=x_test/255.0
class customdataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self, index):
    return self.features[index],self.labels[index]
training_dataset=customdataset(x_train,y_train)
test_dataset=customdataset(x_test,y_test)
train_loader=DataLoader(training_dataset,batch_size=20,shuffle=True)
test_Loader=DataLoader(test_dataset,batch_size=20,shuffle=False)
class neural_network(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,x):
    return self.model(x)
epoch=100
lr=0.1
model=neural_network(x_train.shape[1])
c=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=lr)
for i in range(epoch):
  total_epoch_loss=0
  for batch_features,batch_labels in train_loader:
    output=model(batch_features)
    loss=c(output,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step
    total_epoch_loss+=loss.item()
  avg_loss=total_epoch_loss/len(train_loader)
model.eval()
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_Loader:
    output=model(batch_features)



2.3080681254466375
2.308068131407102
2.3080681403477987
2.3080681383609774
2.308068139354388
2.3080681443214415
2.308068132400513
2.308068137367566
2.3080681522687274
2.3080681572357813
2.3080681383609774
2.3080681294202803
2.3080681184927623
2.3080681294202803
2.3080681453148526
2.3080681363741555
2.30806814233462
2.308068127433459
2.30806814233462
2.3080681443214415
2.3080681453148526
2.308068131407102
2.3080681612094245
2.30806815524896
2.3080681304136914
2.308068143328031
2.308068126440048
2.308068139354388
2.3080681254466375
2.3080681383609774
2.3080681204795837
2.3080681304136914
2.3080681443214415
2.308068139354388
2.308068126440048
2.3080681482950847
2.308068126440048


KeyboardInterrupt: 